## Using Flair Embedding and Glove over English dataset with 2 categories as a POC due to resources shortage

In [3]:
import pandas as pd
dataset = pd.read_csv('../data/dataset.csv', sep='\t', header=None).applymap(str)
dataset.columns = ["Language","label","text"]
languagesData=[]
loc = 0
languages = dataset[dataset.columns[0]].unique()
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1

In [4]:
len(englishData)
englishData = englishData[1:180]


In [5]:
def lower_words(text):
    text = text.str.lower()
    return text

englishData['text'] = lower_words(englishData['text'])
input_str = englishData['text']
print(englishData.head())


  Language        label                                               text
1  english  advertising  trump asks sec to mull half-year corporate fil...
2  english  advertising  wall st. up on trade hopes, s&p equals longest...
3  english  advertising  asian shares hit one-year low on turkey, china...
4  english  advertising  asian stocks weaken as turkey worries weigh, d...
5  english  advertising  most asian shares edge up after wall st. gains...


In [6]:

import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
def remove_url(text):
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    return text


englishData['text'] = remove_url(englishData['text'])
englishData.head()

,Language,label,text
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."
5,english,advertising,most asian shares edge up after wall st. gains...


In [7]:
def remove_numbers(text):
    text = text.str.replace('\d+', '')
    return text


englishData['text'] = remove_numbers(englishData['text'])
englishData.head()

,Language,label,text
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."
5,english,advertising,most asian shares edge up after wall st. gains...


In [8]:
def remove_punctuations(text):
    text = text.str.replace('[^\w\s]','')
    return text


englishData['text'] = remove_punctuations(englishData['text'])
englishData.head()

,Language,label,text
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...
5,english,advertising,most asian shares edge up after wall st gains ...


In [9]:

def remove_blank_space(col):
    col = col.str.strip()
    col = col.replace('\s+', ' ', regex=True)   
    return col


englishData['text'] = remove_blank_space(englishData.text)
englishData.head()

,Language,label,text
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...
5,english,advertising,most asian shares edge up after wall st gains ...


In [13]:
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
def remove_stop_words(text, stop):    
    text.apply(lambda x: [item for item in x if item not in stop])
    return text

stop = stopwords.words('english')
remove_stop_words(englishData['text'],stop)
englishData.head()

Using TensorFlow backend.


,Language,label,text
6,english,advertising,musk bid for tesla no formal offer no firm dea...
7,english,advertising,wall st rallies on solid earnings uschina trad...
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vws ceo was told about emissions software mont...


In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


porter_stemmer = PorterStemmer()
englishData['text'] = englishData['text'].apply(stem_sentences)
englishData.head()

,Language,label,text
1,english,advertising,trump ask sec to mull halfyear corpor file vs ...
2,english,advertising,wall st up on trade hope sp equal longest bull...
3,english,advertising,asian share hit oneyear low on turkey china wo...
4,english,advertising,asian stock weaken as turkey worri weigh dolla...
5,english,advertising,most asian share edg up after wall st gain but...


In [11]:
englishData = englishData[["label","text"]]
englishData['label'] = '__label__' + englishData['label'].astype(str)


In [12]:

englishData.iloc[0:int(len(englishData)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.8):int(len(englishData)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [17]:
englishData.head()

,label,text
6,__label__advertising,musk bid for tesla no formal offer no firm dea...
7,__label__advertising,wall st ralli on solid earn uschina trade talk...
8,__label__advertising,iran say no opec member can take over it share...
9,__label__advertising,onlin cours in data scienc that could help you...
10,__label__advertising,vw ceo wa told about emiss softwar month befor...


In [14]:
#!pip install flair

    100% |████████████████████████████████| 143kB 1.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/9f/7a/003fa432f1e45625626549726c2fbb7a29baa764e9d1fdb2323a5d779f8a/Deprecated-1.2.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/12/704382c3081df3ae3f9d96fe6afb62efa2fa9749be20c301cd2797fb0b52/hyperopt-0.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/e0/c08d5553b89973d9a240605b9c12404bcf8227590de62bae27acbcfe076b/pytorch_pretrained_bert-0.6.2-py3-none-any.whl
    100% |████████████████████████████████| 676.9MB 64kB/s eta 0:00:011  3% |█▏                              | 25.2MB 2.2MB/s eta 0:04:58    18% |██████                          | 127.0MB 1.8MB/s eta 0:05:06    19% |██████▎                         | 133.0MB 1.4MB/s eta 0:06:34    21% |██████▊                         | 142.3MB 1.8MB/s eta 0:05:01    22% |███████▎                        | 153.2MB 1.7MB/s eta 0:05:01    28% |█████████             

  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
    100% |████████████████████████████████| 71kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 5.5MB 1.4MB/s ta 0:00:011
  Stored in directory: /home/kamel/.cache/pip/wheels/49/35/7c/7faf4dc02cdad88d0388e0f895621a34479e1f17f9e4f4ee12
  Stored in directory: /home/kamel/.cache/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built regex smart-open


In [15]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings,DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-06-07 07:22:02,165 Reading data from .
2019-06-07 07:22:02,165 Train: train.csv
2019-06-07 07:22:02,166 Dev: dev.csv
2019-06-07 07:22:02,167 Test: test.csv


/home/kamel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  
/home/kamel/anaconda3/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/kamel/anaconda3/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/kamel/anaconda3/lib/python3.7/site-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' i

In [16]:
len(corpus.dev)

18

In [17]:
word_embeddings = [WordEmbeddings('glove'),FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

2019-06-07 07:22:11,868 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp2w8v8jwf


100%|██████████| 160000128/160000128 [01:43<00:00, 1547940.86B/s]

2019-06-07 07:23:55,516 copying /tmp/tmp2w8v8jwf to cache at /home/kamel/.flair/embeddings/glove.gensim.vectors.npy


2019-06-07 07:23:55,693 removing temp file /tmp/tmp2w8v8jwf
2019-06-07 07:23:55,995 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpb0p_j0us


100%|██████████| 21494764/21494764 [00:13<00:00, 1569963.94B/s]

...

100%|██████████| 19689779/19689779 [00:12<00:00, 1530396.94B/s]

2019-06-07 07:24:24,330 copying /tmp/tmp4132jtpk to cache at /home/kamel/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2019-06-07 07:24:24,359 removing temp file /tmp/tmp4132jtpk


2019-06-07 07:24:24,706 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpuwki4iap


100%|██████████| 19689779/19689779 [00:13<00:00, 1313288.99B/s]

2019-06-07 07:24:38,249 copying /tmp/tmpuwki4iap to cache at /home/kamel/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2019-06-07 07:24:38,275 removing temp file /tmp/tmpuwki4iap


In [18]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [19]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


2019-06-07 07:25:40,126 {'advertising', 'agriculture'}


In [20]:
trainer = ModelTrainer(classifier, corpus)


In [21]:
trainer.train('./', max_epochs=10, patience=1)

2019-06-07 07:25:46,698 ----------------------------------------------------------------------------------------------------
2019-06-07 07:25:46,700 Evaluation method: MICRO_F1_SCORE
2019-06-07 07:25:46,853 ----------------------------------------------------------------------------------------------------
2019-06-07 07:26:56,878 epoch 1 - iter 0/5 - loss 0.72180063
2019-06-07 07:28:56,936 epoch 1 - iter 1/5 - loss 0.69885233
2019-06-07 07:30:02,982 epoch 1 - iter 2/5 - loss 0.67587783
2019-06-07 07:31:38,795 epoch 1 - iter 3/5 - loss 0.65979837
2019-06-07 07:32:31,116 epoch 1 - iter 4/5 - loss 0.63878593
2019-06-07 07:32:31,335 ----------------------------------------------------------------------------------------------------
2019-06-07 07:32:31,336 EPOCH 1 done: loss 0.6388 - lr 0.1000 - bad epochs 0
2019-06-07 07:33:10,875 DEV : loss 1.2981122732162476 - score 0.0
2019-06-07 07:34:09,504 TEST : loss 1.3780931234359741 - score 0.0
2019-06-07 07:34:12,750 ----------------------------

2019-06-07 08:53:42,090 0.2778	0.2778	0.2778
2019-06-07 08:53:42,092 
MICRO_AVG: acc 0.1613 - f1-score 0.2778
MACRO_AVG: acc 0.1389 - f1-score 0.2174
advertising tp: 0 - fp: 13 - fn: 0 - tn: 5 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
agriculture tp: 5 - fp: 0 - fn: 13 - tn: 0 - precision: 1.0000 - recall: 0.2778 - accuracy: 0.2778 - f1-score: 0.4348
2019-06-07 08:53:42,093 ----------------------------------------------------------------------------------------------------


{'test_score': 0.2778,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.3889,
  0.2778,
  0.4444,
  0.3889,
  0.5,
  0.3333,
  0.1667],
 'train_loss_history': [0.6387859344482422,
  0.5977002799510955,
  0.5388383269309998,
  0.5642131745815278,
  0.47355327010154724,
  0.46558079719543455,
  0.5005675256252289,
  0.45463196039199827,
  0.43566877841949464,
  0.4590370118618011],
 'dev_loss_history': [tensor(1.2981),
  tensor(1.3287),
  tensor(1.5225),
  tensor(0.8037),
  tensor(0.9108),
  tensor(0.8114),
  tensor(0.8163),
  tensor(0.7260),
  tensor(0.9011),
  tensor(0.9354)]}